<a href="https://colab.research.google.com/github/Avazork/NN/blob/main/tweets_convolution_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Начнём с уже знакомой загрузки данных.

In [1]:
!git clone https://github.com/Avazork/NN.git

Cloning into 'NN'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 26 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.


Сначала, конечно, загрузим модель fast text, предобученную для нашего материала.

In [2]:
!wget -P /root/input/ -c "http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin"

--2023-01-16 05:09:50--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 178.63.27.41
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin [following]
--2023-01-16 05:09:50--  https://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3417475450 (3.2G) [application/octet-stream]
Saving to: ‘/root/input/ft_native_300_ru_twitter_nltk_word_tokenize.bin’

ft_native_300_ru_tw 100%[===================>]   3.18G  23.2MB/s    in 2m 20s  

2023-01-16 05:12:11 (23.2 MB/s) - ‘/root/input/ft_native_300_ru_twitter_nltk_word_tokenize.bin’ saved [34174

In [3]:
from gensim.models.fasttext import FastText
fasttext = FastText()
fasttext.file_name='/root/input/ft_native_300_ru_twitter_nltk_word_tokenize.bin'
fasttext.load_binary_data()

А вот теперь сами данные.

In [9]:
from lxml import etree
import csv
import numpy as np
from typing import List, Tuple
import nltk
nltk.download('punkt')
from nltk import word_tokenize


def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels


texts,labels=load_sentirueval_2016('/content/NN/bank_train_2016.xml')
label_to_num={'negative':0,'neutral':1,'positive':2}
num_labels=[label_to_num[label] for label in labels]
from sklearn.model_selection import train_test_split
import random
RANDOM_SEED = 23
random.seed(RANDOM_SEED)
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, num_labels, test_size=0.1, random_state=RANDOM_SEED)
vocab=[]
for text in texts:
  tokenized_text=word_tokenize(text)
  for word in tokenized_text:
    if word not in vocab:
      vocab.append(word)
embedding_matrix=np.zeros((len(vocab)+1, 100))
for i,word in enumerate(vocab):
  embedding_matrix[i,0:100]=fasttext[word]
max_length=max([len(word_tokenize(text)) for text in texts])
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
encoded_docs_train = tokenizer.texts_to_sequences(train_texts)
X_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
y_train=np.asarray(train_labels)
encoded_docs_val = tokenizer.texts_to_sequences(val_texts)
X_val = pad_sequences(encoded_docs_val, maxlen=max_length, padding='post')
y_val=np.asarray(val_labels)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-9-b5f281e7850a>:60: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_matrix[i,0:100]=fasttext[word]


In [10]:
X_train.shape,y_train.shape

((8452, 46), (8452,))

In [11]:
X_val.shape,y_val.shape

((940, 46), (940,))

In [12]:
train_labels.count(0),train_labels.count(1), train_labels.count(2), len(y_train)

(1561, 6280, 611, 8452)

In [13]:
X_train
X_val

array([[   11,    60,    57, ...,     0,     0,     0],
       [   60,    57,    17, ...,     0,     0,     0],
       [17512,    17,  2820, ...,     0,     0,     0],
       ...,
       [ 3335, 18630,     8, ...,     0,     0,     0],
       [  177,     4,    92, ...,     0,     0,     0],
       [   11, 11254,    14, ...,     0,     0,     0]], dtype=int32)

А теперь дело за тем, чтобы создать свёрточную нейросеть. (Пока что об аугментации речь не заходит.) Будем использовать экспоненциальное снижение скорости обучения, и это, к слову, уже первый повод поиграться с параметрами.

In [15]:
import tensorflow as tf
tf.random.set_seed(RANDOM_SEED)
from tensorflow.keras.layers import Flatten, Conv1D, MaxPooling1D, SpatialDropout1D,Dense,Dropout,Embedding
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential
from keras.initializers import he_uniform, glorot_uniform
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [16]:
logreg_schedule = ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=20000,
    decay_rate=0.96
)

In [17]:
ConvolutionNet = Sequential()
ConvolutionNet.add(Embedding(len(vocab)+1, 100,weights=[embedding_matrix], input_length=max_length, trainable=False))
ConvolutionNet.add(Conv1D(64, 2, padding='valid', activation='relu',
               kernel_initializer=he_uniform(seed=RANDOM_SEED), name='Conv_B1_L1'))
ConvolutionNet.add(Conv1D(64, 2, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
               name='Conv_B1_L2'))
ConvolutionNet.add(MaxPooling1D(pool_size=2, name='MP1'))
ConvolutionNet.add(SpatialDropout1D(rate=0.15, name='SD1', seed=RANDOM_SEED))

ConvolutionNet.add(Conv1D(64, 3, padding='valid', activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_B2_L1'))
ConvolutionNet.add(Conv1D(64, 3, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_B2_L2'))
ConvolutionNet.add(MaxPooling1D(pool_size=2, name='MP2'))
ConvolutionNet.add(SpatialDropout1D(rate=0.15, name='SD2', seed=RANDOM_SEED))

ConvolutionNet.add(Flatten())
ConvolutionNet.add(Dense(512, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED), name='HL'))
ConvolutionNet.add(Dropout(rate=0.5, seed=RANDOM_SEED, name='DAH'))
ConvolutionNet.add(Dense(3, activation='softmax', kernel_initializer=glorot_uniform(seed=RANDOM_SEED), name='OL'))
ConvolutionNet.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
ConvolutionNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 46, 100)           2165600   
                                                                 
 Conv_B1_L1 (Conv1D)         (None, 45, 64)            12864     
                                                                 
 Conv_B1_L2 (Conv1D)         (None, 44, 64)            8256      
                                                                 
 MP1 (MaxPooling1D)          (None, 22, 64)            0         
                                                                 
 SD1 (SpatialDropout1D)      (None, 22, 64)            0         
                                                                 
 Conv_B2_L1 (Conv1D)         (None, 20, 64)            12352     
                                                                 
 Conv_B2_L2 (Conv1D)         (None, 18, 64)            1

In [21]:
ConvolutionNet.fit(
    X_train, y_train,
    batch_size=64,
    validation_data=(X_val, y_val),
    shuffle=True, epochs=75,
    callbacks=[
        EarlyStopping(
            monitor='val_loss', patience=7, restore_best_weights=True, verbose=1
        )
    ],
    verbose=1)   

Epoch 1/75
133/133 [==============================] - 9s 70ms/step - loss: 0.4604 - sparse_categorical_accuracy: 0.8174 - val_loss: 0.4689 - val_sparse_categorical_accuracy: 0.7936
Epoch 2/75
133/133 [==============================] - 6s 42ms/step - loss: 0.4271 - sparse_categorical_accuracy: 0.8319 - val_loss: 0.4538 - val_sparse_categorical_accuracy: 0.8021
Epoch 3/75
133/133 [==============================] - 9s 66ms/step - loss: 0.3870 - sparse_categorical_accuracy: 0.8454 - val_loss: 0.5076 - val_sparse_categorical_accuracy: 0.7872
Epoch 4/75
133/133 [==============================] - 9s 65ms/step - loss: 0.3673 - sparse_categorical_accuracy: 0.8560 - val_loss: 0.4300 - val_sparse_categorical_accuracy: 0.8255
Epoch 5/75
133/133 [==============================] - 6s 46ms/step - loss: 0.3292 - sparse_categorical_accuracy: 0.8697 - val_loss: 0.4243 - val_sparse_categorical_accuracy: 0.8277
Epoch 6/75
133/133 [==============================] - 8s 57ms/step - loss: 0.3146 - sparse_cate

Видим не самую худшую точность, однако оценим качество, как полагается, вдруг там есть подводные камни?

In [24]:
!git clone https://github.com/mokoron/sentirueval

Cloning into 'sentirueval'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 10), reused 12 (delta 4), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [27]:
test_texts,test_labels=load_sentirueval_2016('/content/sentirueval/banks_test_etalon.xml')
test_num_labels=[label_to_num[label] for label in test_labels]
encoded_docs_test = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
y_test=np.asarray(test_num_labels)
X_test

array([[ 64,   4,  25, ...,   0,   0,   0],
       [ 11,   2,   1, ...,   0,   0,   0],
       [ 64,   4,  25, ...,   0,   0,   0],
       ...,
       [  8,   7,  12, ...,   0,   0,   0],
       [ 60,  57, 173, ...,   0,   0,   0],
       [ 60,  57,  89, ...,   0,   0,   0]], dtype=int32)

In [28]:
y_pred = np.argmax(ConvolutionNet.predict(X_test, batch_size=128), axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=label_to_num.keys(), digits=4))
from sklearn.metrics import f1_score
macro=f1_score(test_num_labels,y_pred,average='macro',labels=[0,2])
micro=f1_score(test_num_labels,y_pred,average='micro',labels=[0,2])
macro,micro

26/26 [==============================] - 1s 24ms/step
              precision    recall  f1-score   support

    negative     0.5408    0.4316    0.4801       767
     neutral     0.7491    0.8298    0.7874      2238
    positive     0.1892    0.1364    0.1585       308

    accuracy                         0.6731      3313
   macro avg     0.4930    0.4659    0.4753      3313
weighted avg     0.6488    0.6731    0.6578      3313



(0.3192742895453364, 0.3907805133577789)

Ну что сказать - попробуем спасти ситуацию. Аугментацией, конечно. Есть два способа - опечатками и синонимами. Естественно, нужно попробовать оба.

In [30]:
!pip install textattack
!pip install tensorflow_text
from textattack.transformations import WordSwapRandomCharacterDeletion
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import CompositeTransformation
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.augmentation import Augmenter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [31]:
class Ochepyatka(WordSwapQWERTY):
  def __init__(
        self, random_one=True, skip_first_char=True, skip_last_char=True, **kwargs
    ):
        super().__init__(**kwargs)
        self.random_one = random_one
        self.skip_first_char = skip_first_char
        self.skip_last_char = skip_last_char

        self._keyboard_adjacency = {
            "ё":["е"],
            "й": ["ц","ф","ё"],
            "ц": ["й", "у", "ф", "ы", "в"],
            "у": ["ц", "ы", "в", "а", "к"],
            "к": ["у", "в", "а", "п", "е"],
            "е": ["к", "а", "п", "р", "н"],
            "н": ["е", "п", "р", "о", "г"],
            "г": ["н", "р", "о", "л", "ш"],
            "ш": ["г", "о", "л", "д", "щ"],
            "щ": ["ш", "л", "д", "з"],
            "з": ["щ", "д","ж","э","х"],
            "х": ["з", "ж","э","ъ"],
            "ъ": ["х", "ж","э"],
            "ф": ["й", "ц", "ы", "я", "ч"],
            "ы": ["й", "ц", "у", "а", "в", "я", "ч"],
            "в": ["ц", "у", "к", "а", "с", "ч", "ы"],
            "а": ["у", "к", "е", "п", "м", "с", "в"],
            "п": ["к", "е", "н", "р", "и", "м", "в"],
            "р": ["е", "н", "г", "п", "о", "и", "т"],
            "о": ["н", "г", "ш", "л", "ь", "т", "р"],
            "л": ["г", "ш", "щ", "д", "ь", "о"],
            "д": ["ш", "щ", "з", "л","ж","ю","б"],
            "ж": ["д", "щ", "з", "х","э","ю"],
            "э": ["ж", "з", "х", "ъ","ю"],
            "я": ["ф", "ы", "ч"],
            "ч": ["ы", "в", "я", "с"],
            "с": ["ч", "в", "а", "м"],
            "м": ["с", "а", "п", "и"],
            "и": ["м", "п", "р", "т"],
            "т": ["и", "р", "о", "ь"],
            "ь": ["т", "о", "л","б"],
            "б": ["ь", "л", "д","ю"],
            "ю": ["б", "д", "ж"]
        }
  def _get_adjacent(self, s):
        s_lower = s.lower()
        if s_lower in self._keyboard_adjacency:
            adjacent_keys = self._keyboard_adjacency[s_lower]
            if s.isupper():
                return [key.upper() for key in adjacent_keys]
            else:
                return adjacent_keys
        else:
            return []

  def _get_replacement_words(self, word):
        if len(word) <= 1:
            return []

        candidate_words = []

        start_idx = 1 if self.skip_first_char else 0
        end_idx = len(word) - (1 + self.skip_last_char)

        if start_idx >= end_idx:
            return []

        if self.random_one:
            i = random.randrange(start_idx, end_idx + 1)
            candidate_word = (
                word[:i] + random.choice(self._get_adjacent(word[i])) + word[i + 1 :]
            )
            candidate_words.append(candidate_word)
        else:
            for i in range(start_idx, end_idx + 1):
                for swap_key in self._get_adjacent(word[i]):
                    candidate_word = word[:i] + swap_key + word[i + 1 :]
                    candidate_words.append(candidate_word)

        return candidate_words

In [33]:
typo_transformation = CompositeTransformation([WordSwapRandomCharacterDeletion(),Ochepyatka(random_one=False)])
typo_constraints = [RepeatModification(),StopwordModification(language='russian')]
typo_augmenter = Augmenter(transformation=typo_transformation, constraints=typo_constraints, pct_words_to_swap=0.5, transformations_per_example=5)
s = texts[1]
typo_augmenter.augment(s)

['Мгение о крежитной курте втб 2 http://t.c/SBJTcsqjCg',
 'Мнегие о коедитной карье втб 24 http://t.o/SBJTcqjCg',
 'Мнение о кредттной карре втб 2 htp://t.co/SBJTcsqjC',
 'Мнепие о креюитной карье втб 2 http://t.co/SBJTcsjCg',
 'Мнкние о кредитнрй катте втб 2 http://t.o/SBJTcsqjCg']

In [39]:
!pip3 install tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
texts_with_typos=[]
labels_of_texts_with_typos=[]
for indx,text in enumerate(tqdm(train_texts)):
    texts_with_typos.extend(typo_augmenter.augment(text))
    labels_of_texts_with_typos.extend([train_labels[indx]]*5)

100%|██████████| 8452/8452 [1:35:14<00:00,  1.48it/s]


In [46]:
with open('ochepyatki.csv', 'w', encoding='utf-8') as f1:
    writer = csv.DictWriter(f1,fieldnames=['text','label'])
    writer.writeheader()
    for i,text in enumerate(texts):
        writer.writerow({'text':text,'label':labels_of_texts_with_typos[i]})

print(len(labels_of_texts_with_typos),len(texts_with_typos))

42260 42260


In [47]:
with open('ochepyatki.csv', 'r', encoding='utf-8') as file:
  texts_with_typos=[]
  labels_of_texts_with_typos=[]
  reader = csv.DictReader(file)
  for row in reader:
    if row!='':
      texts_with_typos.append(row['text'])
      labels_of_texts_with_typos.append(int(row['label']))

In [57]:
pos_ochepyatki=[]
pos_ochepyatki_labels=[]
for text_idx,text in enumerate(texts_with_typos):
  if labels_of_texts_with_typos[text_idx]==2:
    pos_ochepyatki.append(text)
    pos_ochepyatki_labels.append(labels_of_texts_with_typos[text_idx])

neg_ochepyatki=[]
neg_ochepyatki_labels=[]
for text_idx,text in enumerate(texts_with_typos):
  if labels_of_texts_with_typos[text_idx]==0:
    neg_ochepyatki.append(text)
    neg_ochepyatki_labels.append(labels_of_texts_with_typos[text_idx])

pos_neg_ochep=[]
pos_neg_ochep_labels=[]
for text_idx,text in enumerate(texts_with_typos):
  if labels_of_texts_with_typos[text_idx]!=1:
    pos_neg_ochep.append(text)
    pos_neg_ochep_labels.append(labels_of_texts_with_typos[text_idx])

Если с опечатками мы вывернулись, надстроив удобный нам класс из имеющихся, то с синонимами так не получится - придётся брать из ворднета.

In [49]:
!pip install spacy_udpipe
!pip install wiki_ru_wordnet
import spacy_udpipe
spacy_udpipe.download("ru") 
nlp = spacy_udpipe.load("ru")
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()
from nltk.corpus import stopwords
nltk.download('stopwords')
rus_stopwords=stopwords.words('russian')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Already downloaded a model for the 'ru' language


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
def total_syn_aug(text,n:int)->List[str]:
  sourc=nlp(text.lower())
  lemmatext=[wf.lemma_ for wf in sourc]
  result=[]
  for i in range(n):
    changed_text=[]
    for word in lemmatext:
      if word not in rus_stopwords:
        synsets = wikiwordnet.get_synsets(word)
        if synsets:
          synset=random.choice(synsets)
          words=synset.get_words()
          new_word=random.choice(list(words))
          if new_word.lemma()!=word:
            changed_text.append(new_word.lemma())
        else:
         changed_text.append(word)
      else:
        changed_text.append(word)
    if changed_text!=lemmatext:
      result.append(' '.join(changed_text))
  return set(result)

In [53]:
texts_with_synonyms=[]
labels_of_texts_with_synonyms=[]
for text_idx,text in enumerate(tqdm(train_texts)):
  if train_labels[text_idx]==2:
    augmented_texts=total_syn_aug(text,5)
    texts_with_synonyms.extend(augmented_texts)
    labels_of_texts_with_synonyms.extend([train_labels[text_idx]]*len(augmented_texts))
  '''elif train_labels[text_idx]==0:
    augmented_texts=augment_with_synonyms(text,3)
    texts_with_synonyms.extend(augmented_texts)
    labels_of_texts_with_synonyms.extend([train_labels[text_idx]]*len(augmented_texts))'''

100%|██████████| 8452/8452 [00:14<00:00, 583.54it/s]


In [55]:
print(len(texts_with_synonyms),len(labels_of_texts_with_synonyms))
texts_with_synonyms[:20]

1991 1991


['и втб24 поднимать процент по . госбанка догонять : 23 повысить по картина - -',
 'и втб24 процент по . госбанка догонять : 23 повысить по сорт - -',
 'и втб24 поднимать процент по . госбанка догонять : 23 повысить по - -',
 'и втб24 процент по . госбанка догонять торг : 23 повысить по картина - -',
 'обновить r mobile от @raiffeisen_ru и больно . в предшествующий не , но сейчас без хула .',
 'обновить r mobile от @raiffeisen_ru и зело . в прошлый не , но сейчас без порицание .',
 'обновить r mobile от @raiffeisen_ru и чересчур . в прошлый редакция не доставать , но сейчас без укор .',
 'обновить r mobile от @raiffeisen_ru и чрезвычайно пьяный . в прошлый не , но сейчас без укор .',
 'обновить r mobile от @raiffeisen_ru и крайне . в не , но сейчас без .',
 'да ! я же не ! пенсия я быть с сбербанок .',
 'да ! я же не ! пенсия я быть накоплять с сбербанок .',
 'россия обновить сайт http://t.co/343h6e1cek',
 'россия обновить http://t.co/343h6e1cek',
 'обновить : http://t.co/eis9wiw7ls',


Не такой хороший буст, как от опечаток, но тоже ничего. Далее планировалось включить аугментацию с помощью берта, но произошло то, чего, возможно, следовало ожидать (хотя я не ожидал): даже несущественный результат требовал нескольких дней ожидания. Ну а теперь проверка качества - перетусовка на фолды, расширение нашими синонимами и само обучение.

In [58]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, num_labels, test_size=0.1, random_state=RANDOM_SEED)
train_texts.extend(texts_with_synonyms)
train_labels.extend(labels_of_texts_with_synonyms)
train_texts.extend(pos_neg_ochep)
train_labels.extend(pos_neg_ochep_labels)
train_labels.count(0),train_labels.count(1), train_labels.count(2), len(y_train)

(3211, 6280, 3347, 8452)

In [59]:
encoded_docs_train = tokenizer.texts_to_sequences(train_texts)
X_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
y_train=np.asarray(train_labels)

In [60]:
Convolution2 = Sequential()
Convolution2.add(Embedding(len(vocab)+1, 100,weights=[embedding_matrix], input_length=max_length, trainable=False))
Convolution2.add(Conv1D(64, 3, padding='valid', activation='relu',
               kernel_initializer=he_uniform(seed=RANDOM_SEED), name='Conv_B1_L1'))
Convolution2.add(Conv1D(64, 3, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
               name='Conv_B1_L2'))
Convolution2.add(MaxPooling1D(pool_size=2, name='MP1'))
Convolution2.add(SpatialDropout1D(rate=0.15, name='SD1', seed=RANDOM_SEED))

Convolution2.add(Conv1D(64, 4, padding='valid', activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_B2_L1'))
Convolution2.add(Conv1D(64, 4, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_B2_L2'))
Convolution2.add(MaxPooling1D(pool_size=2, name='MP2'))
Convolution2.add(SpatialDropout1D(rate=0.15, name='SD2', seed=RANDOM_SEED))

Convolution2.add(Flatten())
Convolution2.add(Dense(512, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED), name='HL'))
Convolution2.add(Dropout(rate=0.5, seed=RANDOM_SEED, name='DAH'))
Convolution2.add(Dense(3, activation='softmax', kernel_initializer=glorot_uniform(seed=RANDOM_SEED), name='OL'))
Convolution2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
Convolution2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 46, 100)           2165600   
                                                                 
 Conv_B1_L1 (Conv1D)         (None, 44, 64)            19264     
                                                                 
 Conv_B1_L2 (Conv1D)         (None, 42, 64)            12352     
                                                                 
 MP1 (MaxPooling1D)          (None, 21, 64)            0         
                                                                 
 SD1 (SpatialDropout1D)      (None, 21, 64)            0         
                                                                 
 Conv_B2_L1 (Conv1D)         (None, 18, 64)            16448     
                                                                 
 Conv_B2_L2 (Conv1D)         (None, 15, 64)           

In [61]:
Convolution2.fit(
    X_train, 
    y_train, 
    batch_size=64, 
    validation_data=(X_val, y_val), 
    shuffle=True, 
    epochs=75, 
    callbacks=[EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)], 
    verbose=1)

Epoch 1/75
201/201 [==============================] - 9s 39ms/step - loss: 0.9758 - sparse_categorical_accuracy: 0.5608 - val_loss: 0.7064 - val_sparse_categorical_accuracy: 0.7585
Epoch 2/75
201/201 [==============================] - 8s 39ms/step - loss: 0.8765 - sparse_categorical_accuracy: 0.6298 - val_loss: 0.7361 - val_sparse_categorical_accuracy: 0.7489
Epoch 3/75
201/201 [==============================] - 8s 39ms/step - loss: 0.8099 - sparse_categorical_accuracy: 0.6743 - val_loss: 0.7095 - val_sparse_categorical_accuracy: 0.7447
Epoch 4/75
201/201 [==============================] - 11s 55ms/step - loss: 0.7536 - sparse_categorical_accuracy: 0.7053 - val_loss: 0.5942 - val_sparse_categorical_accuracy: 0.7979
Epoch 5/75
201/201 [==============================] - 10s 51ms/step - loss: 0.7148 - sparse_categorical_accuracy: 0.7204 - val_loss: 0.5887 - val_sparse_categorical_accuracy: 0.8064
Epoch 6/75
201/201 [==============================] - 8s 37ms/step - loss: 0.6875 - sparse_ca

In [64]:
y_pred = np.argmax(Convolution2.predict(X_test, batch_size=128), axis=-1)

26/26 [==============================] - 1s 17ms/step


In [65]:
print(classification_report(y_test, y_pred, target_names=label_to_num.keys(), digits=4))

              precision    recall  f1-score   support

    negative     0.4925    0.1721    0.2551       767
     neutral     0.6942    0.7730    0.7315      2238
    positive     0.0976    0.1753    0.1254       308

    accuracy                         0.5783      3313
   macro avg     0.4281    0.3735    0.3707      3313
weighted avg     0.5921    0.5783    0.5649      3313



In [66]:
macro=f1_score(test_num_labels,y_pred,average='macro',labels=[0,2])
micro=f1_score(test_num_labels,y_pred,average='micro',labels=[0,2])
macro,micro
     

(0.19025400191890118, 0.1962025316455696)

Интересные дела! У нас здорово просели практически все показатели. С чем это может быть связано? Возможно, замена синонимами привела к тому, что какие-то явно стилистически маркированные слова заменялись на нейтральные или по-другому окрашенные, и поэтому понизилось качество работы (ведь предыдущее задание показало, что не самой плохой идеей является просто в лоб считать стилистическую маркировку лексики). Может, если бы мы аугментировали синонимами с помощью берта, жить стало бы лучше, но лишней пары недель с постоянно включенным компьютером, увы, в запасе нет. Трудно предположить, как на такую просадку повлияли опечатки. В общем, факт остаётся фактом.